In [1]:
import MySQLdb

In [2]:
import requests
import MySQLdb 
import pandas as pd
from dotenv import load_dotenv
import os
import time
import datetime
import pytz
from datetime import timedelta

UTC = pytz.utc
IST = pytz.timezone('Asia/Kolkata')

class Db:
    def __init__(self,db, host, user, password):
        self.host = host
        self.user = user
        self.password = password
        self.db = MySQLdb.connect(host=self.host, user=self.user, password=self.password, db=db)
        self.cursor = self.db.cursor()

load_dotenv("ig.env")

if os.getenv("DEBUGGING")=="true":
    insert_db_host = os.getenv("DB_HOST_LOCAL")
    insert_db_user = os.getenv("DB_USER_LOCAL")
    insert_db_password = os.getenv("DB_PASSWORD_LOCAL")

    fetch_db_host= os.getenv("DB_HOST_MAIN")
    fetch_db_user = os.getenv("DB_USER_MAIN")
    fetch_db_password = os.getenv("DB_PASSWORD_MAIN")
else:
    insert_db_host = os.getenv("DB_HOST_MAIN")
    insert_db_user = os.getenv("DB_USER_MAIN")
    insert_db_password = os.getenv("DB_PASSWORD_MAIN")

    fetch_db_host= os.getenv("DB_HOST_MAIN")
    fetch_db_user = os.getenv("DB_USER_MAIN")
    fetch_db_password = os.getenv("DB_PASSWORD_MAIN")
    


fetchdb = Db("scrap_token",fetch_db_host,fetch_db_user,fetch_db_password)
fetchdb_cursor = fetchdb.cursor
insertdb = Db("Instagram_Scraping",insert_db_host,insert_db_user,insert_db_password)
insertdb_cursor = insertdb.cursor
table_name = "ig_insights"
frequency = "daily"



def page_id_tkn():
    sql = "SELECT page_id,long_token FROM Token_instapagetoken"
    fetchdb_cursor.execute(sql)
    fetchdb_data = fetchdb_cursor.fetchall()
    return fetchdb_data

page_id_tkn_list = page_id_tkn()
page_id_tkn_dict = {page_id_tkn_list[i][0]:page_id_tkn_list[i][1] for i in range(len(page_id_tkn_list))}

def month_ends_date():
    current_month = datetime.datetime.now(IST).month
    last_month = current_month - 1
    if last_month == 0:
        last_month = 12
        last_month_first_date = datetime.datetime(datetime.datetime.now(IST).year - 1, last_month, 1, 0, 0, 0).strftime('%Y-%m-%d')
    else:
        last_month_first_date = datetime.datetime(datetime.datetime.now(IST).year, last_month, 1, 0, 0, 0).strftime('%Y-%m-%d')

    current_month_first_date = datetime.datetime(datetime.datetime.now(IST).year, current_month, 1, 0, 0, 0).strftime('%Y-%m-%d')

    return last_month_first_date,current_month_first_date

def start_end_frequency(frequency):
    start,end,scrap_type = None,None,None
    end = datetime.datetime.now(IST)
    if frequency == "daily":
        start = (end - timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")
        end = end.strftime("%Y-%m-%d %H:%M:%S")
        scrap_type= 'D'
    elif frequency == "weekly":
        start = (end - timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S")
        end = end.strftime("%Y-%m-%d %H:%M:%S")
        scrap_type= 'W'
    elif frequency=="monthly":
        start,end = month_ends_date()
        scrap_type= 'M'
    elif frequency=="manual":
        print("The code for this category is still in progress.")
    else:
        print("Invalid String")
    return start,end,scrap_type

start,end,scrap_type = start_end_frequency(frequency)

start,end = start.split()[0],end.split()[0]
#print(start,end)    

def page_post_id():
    sql = "SELECT page_id,post_id,media_type FROM {} where frequency = '{}' AND  date(created_time)>='{}' and date(created_time)<='{}' ".format(table_name,scrap_type,start,end)
    insertdb_cursor.execute(sql)
    pagepostdata = insertdb_cursor.fetchall()
    return pagepostdata

pagepostdata = page_post_id()
field_titles = [ "impressions","reach","engagement","saved","video_views"]

def metric_maker(media_type):
    metric_list = field_titles
    
    if media_type=="IMAGE":
        #metrics=",".join(metric_list[:-1]) 
        metrics = "impressions,reach,engagement,saved"
    elif media_type=="VIDEO":
        #metrics=",".join(metric_list[:])
        metrics = "impressions,reach,engagement,saved,video_views"
    else:
        #metrics = ','.join(["carousel_album_"+x for x in metric_list[:]])  
        metrics = 'carousel_album_impressions,carousel_album_reach,carousel_album_engagement,carousel_album_saved,carousel_album_video_views'
   
    
    return metrics
  
#CAROUSEL_ALBUM
#,
def fetch_insights():
    for page_id,post_id,media_type in pagepostdata:
        try:
            page_token = page_id_tkn_dict[page_id]
            metrics = metric_maker(media_type)
            url = """https://graph.facebook.com/v13.0/{}/insights?access_token={}&metric={}""".format(post_id,page_token,metrics)
            response = requests.get(url)
            post_data = response.json()
            post_metric= {}
            post_metric = {"post_id":post_id,"page_id":page_id,"media_type":media_type}
            for metric in post_data.get("data"):
                title =  metric.get("name")
                if media_type=="CAROUSEL_ALBUM":
                    title = title.replace("carousel_album_","")
                post_metric[title] = metric.get("values")[0].get("value")
            if media_type=="IMAGE":
                post_metric["video_views"]=0 
            for field in field_titles:
                if field not in list(post_metric.keys()):
                    post_metric[field]= 0
            post_metric['updated_time'] = datetime.datetime.now(IST).strftime("%Y-%m-%d %H:%M:%S")
            
            insert_insights(post_metric,page_id,post_id)
        except Exception as e:
            print(e)
            
        
def insert_insights(post_metric,page_id,post_id):
    idb = insertdb.db
    columns = list(post_metric.keys())
    values = list(post_metric.values())
    
    helper = ['%s'] * len(columns)
    duplicate_update_sql = ', '.join(['%s=VALUES(%s)' % (k, k) for k in columns])
    try:
        insertdb_cursor.execute(
            """INSERT INTO {} ({}) VALUES ({}) ON DUPLICATE KEY UPDATE {} """.format(table_name, ' ,'.join(columns)  , ' ,'.join(helper), duplicate_update_sql),
            values)
        idb.commit()
        print(page_id,post_id,"Success")
    except Exception as e:
        print(e)
        idb.rollback()
print("Started at",datetime.datetime.now(IST).strftime("%Y-%m-%d %H:%M:%S"))

fetch_insights()

print("Ended at",datetime.datetime.now(IST).strftime("%Y-%m-%d %H:%M:%S")) 

TypeError: connect() argument 1 must be str, not None

In [ ]:
def move1(str1, str2):
    char = str1[0]
    str2 = str2 + char
    str1 = str1[1:]
    return str1, str2
def move2(str2, str3):
    char = str2[-1]
    str3 = char + str2
    str2 = str2[:-1]
    return str2, str3
    
    
    
    
str1 = 'akbjofjfhao'
str2 = ""
str3 = ""
new_str = ""
n = len(str1)
for i in range(n):
    if str1 == "" and str2 == "":
        break
    elif str1[i] == min(str1):
        str2, str3 = move1(str2, str3)
        str1, str2 = move1(str1, str2)
    else:
        str1, str2 = move1(str1, str2)
    
    
    

In [2]:
str1 = "hello"
print(min(str1))

e


In [3]:
def distance(x, y):
    return ((x[0] - y[0])**2 + (x[1] - y[1])**2)**.5

class:
    def func(self, x, y, z)
        x = 2
        y = 3
        z = [[0, 0], [2, 0], [0, 2]]

        max_dist = 0
        for i in range(len(z)):
            for j in range(i, len(z)):
                dist = distance(z[i], z[j])
                if dist > max_dist:
                    max_dist = dist
        return (max_dist/x)
        

1.4142135623730951


In [1]:
int.__add__(2, 3)

5

In [17]:
p = 5
count = 1
n = 0
while count<= p:
    ele = 6*n + 2
    if ele%4 == 0:
        p+=1
        count+=1
        continue
    else:
        count+=1
        n += 2
        print(ele)

2
14
26
38
50


In [10]:
s='Python RockS!'
print(s.swapcase().rpartition('o')[2])

pYTHON rOCKs!


In [1]:
a = "test"
set(a)

{'e', 's', 't'}

In [7]:
def minSubstring(a, b, currMin = float("inf")):
    if len(a) < len(b) or (len(a) == len(b) and a!=b) or set(a).intersection(set(b)) != set(b):
        return False
    else:
        return min(len(a), currMin)
    return minSubstring(a[1:], b)

minSubstring("fight", "it")

5